<a href="https://colab.research.google.com/github/kearns-cu/AI_Projects/blob/main/Kearns_Lakka_Naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification of messages as spam or not spam using Naive Bayes algorithm 

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Import Dataset
# Define the file path
file_path = '/tmp/SMS'

# Read the file into a DataFrame
df = pd.read_table(file_path, sep='\t', header=None, names=['label', 'sms_message'])

# Print the first few rows of the DataFrame
print(df.head())

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# map the 'ham' value to 0 and the 'spam' value to 1.
df['label'] = df.label.map({'ham':0,'spam':1})

In [ ]:
# Randomize the dataset
df = df.sample(frac=1, random_state=1)
df

,label,sms_message
1078,0,"Yep, by the pretty sculpture"
4028,0,"Yes, princess. Are you going to make me moan?"
958,0,Welp apparently he retired
4642,0,Havent.
4674,0,I forgot 2 ask ü all smth.. There's a card on ...
...,...,...
905,0,"We're all getting worried over here, derek and..."
5192,0,Oh oh... Den muz change plan liao... Go back h...
3980,0,CERI U REBEL! SWEET DREAMZ ME LITTLE BUDDY!! C...
235,1,Text & meet someone sexy today. U can find a d...


In [ ]:
# Split into training and test sets
training_test_index = round(len(df) * 0.8)

training = df[:training_test_index].reset_index(drop=True)
test = df[training_test_index:].reset_index(drop=True)

print(training.shape)
print(test.shape)

(4458, 2)
(1114, 2)


In [ ]:
#  data cleaning 
training['sms_message'] = training['sms_message'].str.replace('\W', ' ') # Removes punctuation
training['sms_message'] = training['sms_message'].str.lower() ### making all the words lowercase

test['sms_message'] = test['sms_message'].str.replace('\W', ' ') # Removes punctuation
test['sms_message'] = test['sms_message'].str.lower() ### making all the words lowercase
training

<ipython-input-29-7df05edabc8a>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  training['sms_message'] = training['sms_message'].str.replace('\W', ' ') # Removes punctuation
<ipython-input-29-7df05edabc8a>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  test['sms_message'] = test['sms_message'].str.replace('\W', ' ') # Removes punctuation


,label,sms_message
0,0,yep by the pretty sculpture
1,0,yes princess are you going to make me moan
2,0,welp apparently he retired
3,0,havent
4,0,i forgot 2 ask ü all smth there s a card on ...
...,...,...
4453,0,sorry i ll call later in meeting any thing re...
4454,0,babe i fucking love you too you know fuck...
4455,1,u ve been selected to stay in 1 of 250 top bri...
4456,0,hello my boytoy geeee i miss you already a...


In [ ]:
### creating vocabulary from training data
training['sms_message'] = training['sms_message'].str.split()
vocabulary = []
for sms in training['sms_message']:
   for word in sms:
      vocabulary.append(word)
vocabulary = list(set(vocabulary))  ### only count the number of unique words
print(len(vocabulary))

7783


In [ ]:
word_counts_per_sms = {unique_word: [0] * len(training['sms_message']) for unique_word in vocabulary}

for index, sms in enumerate(training['sms_message']):
   for word in sms:
      word_counts_per_sms[word][index] += 1
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts

,abdomen,suganya,tones,09066368753,side,ave,kissing,flowing,kappa,see,...,bpo,difference,fuelled,emotion,gari,pataistha,somewheresomeone,password,access,choose
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4453,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4454,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4455,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4456,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
training_new = pd.concat([training, word_counts], axis=1)
training_new.head()

,label,sms_message,abdomen,suganya,tones,09066368753,side,ave,kissing,flowing,...,bpo,difference,fuelled,emotion,gari,pataistha,somewheresomeone,password,access,choose
0,0,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Laplace smoothing
alpha = 1

# You will start from here.

In [ ]:
# Hints:
# Step 1: you need to caculate P(Spam) and P(Ham)
# Step 2: you need to count N_Spam, N_Ham
# Step 3: you need to count the number of times the word w occurs in spam/ham message: N_w_spam
# Step 4: then you can calculate the prob of occurance of each word: p(w|spam)=(N_w_spam+alpha)/(N_Spam+alpha*N_Vocabulary)
#         p(w|Ham)=(N_w_ham+alpha)/(N_Ham+alpha*N_Vocabulary)
# Step 5: Now do the prediction if it is spam or ham for each sentence, then store your prediction results (1 or 0 ) to test['predicted']

# Notes: for the symbols listed above, please check the last page of lecture. 
#        

In [ ]:
# Training the model
n_spam = training_new[training_new['label'] == 1]['sms_message'].apply(len).sum()
n_ham = training_new[training_new['label'] == 0]['sms_message'].apply(len).sum()

n_vocabulary = len(vocabulary)
spam_messages = training_new[training_new['label'] == 1]
ham_messages = training_new[training_new['label'] == 0]

parameters_spam = {word: (spam_messages[word].sum() + alpha) / (n_spam + alpha * n_vocabulary) for word in vocabulary}
parameters_ham = {word: (ham_messages[word].sum() + alpha) / (n_ham + alpha * n_vocabulary) for word in vocabulary}

In [ ]:
# Making predictions
import numpy as np

def classify(message):
    message = message.lower().replace('\W', ' ').split()
    p_spam_given_message = np.log(p_spam)
    p_ham_given_message = np.log(p_ham)

    for word in message:
        if word in parameters_spam:
            p_spam_given_message += np.log(parameters_spam[word])
        if word in parameters_ham:
            p_ham_given_message += np.log(parameters_ham[word])

    if p_spam_given_message > p_ham_given_message:
        return 1
    else:
        return 0

p_spam = training_new['label'].value_counts(normalize=True)[1]
p_ham = training_new['label'].value_counts(normalize=True)[0]

test['predicted'] = test['sms_message'].apply(classify)

# Calculate accuracy, precision, recall and F1_score. 

In [ ]:
# Model Evaluation 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print('Accuracy score: {}'.format(accuracy_score(test['label'], test['predicted'])))
print('Precision score: {}'.format(precision_score(test['label'], test['predicted'])))
print('Recall score: {}'.format(recall_score(test['label'], test['predicted'])))
print('F1 score: {}'.format(f1_score(test['label'], test['predicted'])))



Accuracy score: 0.9883303411131059
Precision score: 0.9652777777777778
Recall score: 0.9455782312925171
F1 score: 0.9553264604810997


### Now  use function MultinomialNB (from sklearn.naive_bayes import MultinomialNB) to validate your results, check the accuracy

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Concatenate
all_messages = training['sms_message'].apply(' '.join)

# Matrix of word frequencies
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(all_messages)
y_train = training['label']
X_test = vectorizer.transform(test['sms_message'])
y_test = test['label']


# Train classifier
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Predictions
y_pred = clf.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: {:.2f}%'.format(accuracy * 100))


Accuracy using MultinomialNB: 0.8680430879712747
